In [ ]:
country  = 'Russia'

# Introduction

This notebook evaluates several epidemiology models for COVID-19 data.

The primary difference from classical models is that each model includes variable $O$ for _Observed_ cases. Models use the simplest estimation for $O$:

$$\frac{dO}{dt} = k I$$

where $k$ is _reveal efficiency_ coeffitient, showing, how many infected people are identified and positevely tested.

Models have $n$ parameters and $m$ variables in initial vectors. Combined they form $n+m$-dimentional area where we search a solution for IV problem that approximates known cases best. To find the optimal solution we use _Non-linear Least Squares_ method from `scipy.optimize` package (see [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html)).

For every model we present 5 best solutions found. The field `Precision` in the report table is the standard deviation of the solution from data spots.

# The best model

It is expected that the best model for COVID-19 is [SIR with carriers](#sirc)

# TOC
* [Country data](#country)
* [Models with immunity](#r-models)
* * [Basic SIR model](#sir)
* * [Model with carriers](#sirc)
* * [Model with incubation period](#seir)

* [Models without immunity](#s-models)
* * [Basic SIRS model](#sirs)
* * [Model with carriers](#sirsc)
* * [Model with incubation period](#seirs)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit
from scipy.integrate import odeint
from datetime import datetime

%matplotlib inline

In [ ]:
print("Computed on: ", datetime.now().strftime(format='%y-%m-%d %T'))

In [ ]:
import corona

# Country data <a name="country"/>

In [ ]:
info = corona.CovidModel(country)
print("Country: ", info.country)
print("Population: ", info.population)
print("Number of revealed cases: ", info.data[0][-1])
print("Number of deaths: ", info.data[1][-1])

In [ ]:
delta = info.data[:,1:] - info.data[:,:-1]
days = np.arange(np.shape(delta)[1])

fig, ax1 = plt.subplots(figsize=(14,8))

ax1.bar(days, delta[0], width=.8)
ax1.set_ylabel('Cases per day')
ax1.set_xlabel("Days since January 22, 2020")

ax2 = ax1.twinx()
ax2.plot(days, info.data[0][1:], '-', color='red', linewidth=5)
ax2.set_ylabel('Total number of cases')
fig.tight_layout()

plt.title(info.country + ": COVID19 stats");
plt.grid()

Red dots show the total number of confirmed cases. Blue bars represent number of confirmed cases daily.

"Day 0" is January 22, when WHO started collecting data from countries.

# Models with immunity <a name="r-models"/>

## SIR - the basic epidemiology model <a name="sir"/>
This model is modified [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model): 
- **S** is *Susceptible*
- **I** is *Infectious*
- **R** is *Recovered* (and *diseased*). The model expects that this pool becomes 100% immune.
- **O** is *cOnfirmed* individuals, both hospitalized and quorantined at home with a positive test

The model utilises three parameters: $\alpha$,$\beta$ and $k$:
- $\alpha$ models the speed of infection spread,
- $\beta$ represents the speed of recovery (and deaths)
- $k$ is the reveal factor, how many infectious people are confirmed as positive.

The differential equations:<br>
$\frac{dS}{dt} = -\alpha S I$,<br>
$\frac{dI}{dt} = \alpha S I - \beta I$, <br>
$\frac{dR}{dt} = \beta I$,<br>
$\frac{dO}{dt} = k I$.

The initial value vector is ${S_0, I_0, R_0, O_0}$, where $S_0 = 1 - I_0 - R_0$ and $O_0$ is the number of revealed cases at the beginning. The initial values $I_0$ and $R_0$ are guessed during optimization process.

In [ ]:
test_sir = corona.CovidModel(country, model='sir_o_r0')

test_sir.run(500, figsize=(10, 6))

## Part of the population are symptom-less infectious carriers <a name="sirc"/>
**SIRC-O** - a SIR model with carriers and with explicit confirmed cases. Recovered species posess immunity.

This model is modified [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model): 
- **S** is *Succeptible*
- **I** is *Infectious*, i.e. with syptoms of the disease.
- **C** is *Carrier*, i.e. infectious, but without symptoms or actual illness.
- **R** is *Recovered* (and *diseased*). While in **R** state people are immune.
- **O** is *cOnfirmed* individuals, both hospitalized and quorantined at home with a positive test

The major feature of this model is having people carry the virus and infect others without actually being ill.

The model utilises the following parameters: 
- $\alpha$ models the speed of infection spread,
- $\beta$ represents the speed of recovery (and deaths),
- $c_s$ models efficiency of infection transmission by carriers compared to infectious.
- $i_r$ is the ratio of infectious, who recovered (became immune), and $1 - i_r$ become carriers.
- $c_r$ represents the speed of recovery of carriers.
- $c_i$ represents the speed of conversion of carriers to infected.
- $k$ is the reveal factor, how many infectious people and carriers are confirmed as positive.

The differential equations:
$\frac{dS}{dt} = -\alpha S I - c_s \alpha S C$,<br>
$\frac{dI}{dt} = \alpha S I + c_s \alpha S C - \beta I$ + c_i C, <br>
$\frac{dR}{dt} = \beta i_r I$ + c_r C,<br>
$\frac{dC}{dt} = \beta (1 - i_r) I - (c_r + c_i) C$,<br>
$\frac{dO}{dt} = k (I + C)$.

The initial value vector is ${S_0, I_0, R_0, C_0, O_0}$, where $S_0 = 1 - I_0 - R_0 - C_0$, and $O_0$ is the number of revealed cases at the beginning. The initial values $I_0$, $R_0$ and $C_0$ are guessed during optimization process.

In [ ]:
test_sirc = corona.CovidModel(country, model='sir_co')
test_sirc.run(300)

## Model with exposed (incubation period) <a name="seir"/>
    
**SEIR-O** is SIR model with incubation period.

This model is modified [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model): 
- **S** is *Succeptible*
- **E** is *Exposed*, i.e. people who are infected, but not yet infectious.
- **I** is *Infectious*
- **R** is *Recovered* (and *diseased*). While in **R** state people are immune.
- **O** is *cOnfirmed* individuals, both hospitalized and quorantined at home with a positive test

The major feature of this model is conversion of some or recovered back into succeptible.

The model utilises the following parameters: 
- $\alpha$ models the speed of infection spread,
- $\beta$ represents the speed of recovery (and deaths),
- $e_i$ represents the speed of becoming infectious,
- $e_i$ represents the speed of becoming immune without becoming infectious,
- $k$ is the reveal factor, how many infectious people are confirmed as positive.

The differential equations:<br>
$\frac{dS}{dt} = -\alpha S I$,<br>
$\frac{dE}{dt} = \alpha S I - e_i E - e_r E$, <br>
$\frac{dI}{dt} = e_i E - \beta I$, <br>
$\frac{dR}{dt} = \beta I + e_r E$,<br>
$\frac{dO}{dt} = k I$.

The initial value vector is ${S_0, E_0, I_0, R_0, O_0}$, where $S_0 = 1 - E_0 - I_0 - R_0$ and $O_0$ is the number of revealed cases at the beginning. The initial values $E_0$, $I_0$ and $R_0$ are guessed during optimization process.


In [ ]:
test_seir = corona.CovidModel(country, model='seir_o')

test_seir.run(300)

# Models without immunity <a name="s-models"></a>


## Some of recovered become susceptible again <a name="sirs"/>

**SIRS-O** is SIR model without antigen and with explicit confirmed cases

This model is modified [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model): 
- **S** is *Succeptible*
- **I** is *Infectious*
- **R** is *Recovered* (and *diseased*). While in **R** state people are immune.
- **O** is *cOnfirmed* individuals, both hospitalized and quorantined at home with a positive test

The major feature of this model is conversion of some or recovered back into succeptible.

The model utilises the following parameters: 
- $\alpha$ models the speed of infection spread,
- $\beta$ represents the speed of recovery (and deaths),
- $r_s$ is the ratio of recovered people who are susceptible again,
- $k$ is the reveal factor, how many infectious people are confirmed as positive.

The differential equations:<br>
$\frac{dS}{dt} = -\alpha S I + r_s R$,<br>
$\frac{dI}{dt} = \alpha S I - \beta I$, <br>
$\frac{dR}{dt} = \beta I - r_s R$,<br>
$\frac{dO}{dt} = k I$.

The initial value vector is ${S_0, I_0, R_0, O_0}$, where $S_0 = 1 - I_0 - R_0$ and $O_0$ is the number of revealed cases at the beginning. The initial values $I_0$ and $R_0$ are guessed during optimization process.


In [ ]:
test_sirs = corona.CovidModel(country, model='sir_so_r0')
test_sirs.run(300)

## Part of the population are symptom-less infectious carriers, and recovered loose the immunity<a name="sirsc"/>

**SIRSC-O** - a SIRS model with carriers and with explicit confirmed cases. Recovered species turn back into susceptible.

This model is modified [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model): 
- **S** is *Succeptible*
- **I** is *Infectious*, i.e. with syptoms of the disease.
- **C** is *Carrier*, i.e. infectious, but without symptoms or actual illness.
- **R** is *Recovered* (and *diseased*). While in **R** state people are immune.
- **O** is *cOnfirmed* individuals, both hospitalized and quorantined at home with a positive test

The major feature of this model is having people carry the virus and infect others without actually being ill.

The model utilises the following parameters: 
- $\alpha$ models the speed of infection spread,
- $\beta$ represents the speed of recovery (and deaths),
- $c_s$ models efficiency of infection transmission by carriers compared to infectious.
- $i_r$ is the ratio of infectious, who recovered (became immune), and $1 - i_r$ become carriers.
- $c_r$ represents the speed of recovery of carriers.
- $c_i$ represents the speed of conversion of carriers to infected.
- $r_s$  is the ratio of recovered people who are susceptible again. 
- $k$ is the reveal factor, how many infectious people and carriers are confirmed as positive.

The differential equations:
$\frac{dS}{dt} = -\alpha S I - c_s \alpha S C$ + r_s R,<br>
$\frac{dI}{dt} = \alpha S I + c_s \alpha S C - \beta I$ + c_i C, <br>
$\frac{dR}{dt} = \beta i_r I + c_r C - r_s R$,<br>
$\frac{dC}{dt} = \beta (1 - i_r) I - (c_r + c_i) C$,<br>
$\frac{dO}{dt} = k (I + C)$.

The initial value vector is ${S_0, I_0, R_0, C_0, O_0}$, where $S_0 = 1 - I_0 - R_0 - C_0$, and $O_0$ is the number of revealed cases at the beginning. The initial values $I_0$, $R_0$ and $C_0$ are guessed during optimization process.

In [ ]:
test_sir_sco = corona.CovidModel(country, model='sir_sco')
test_sir_sco.run(300)


## Experimental model with carriers and no immunity
The model is almost the same as the previous one. The only difference is in revealing efficiency for infectious and carriers. It expects that infectious and carriers are revealed at different pace.

- $k_i$ is the reveal factor, how many infectious people are confirmed as positive.
- $k_c$ is the reveal factor, how many carriers are confirmed as positive.

The differential equations:<br>
$\frac{dS}{dt} = -\alpha S I - c_s \alpha S C$ + r_s R,<br>
$\frac{dI}{dt} = \alpha S I + c_s \alpha S C - \beta I$ + c_i C, <br>
$\frac{dR}{dt} = \beta i_r I + c_r C - r_s R$,<br>
$\frac{dC}{dt} = \beta (1 - i_r) I - (c_r + c_i) C$,<br>
$\frac{dO}{dt} = k_i I + k_c C$.

The initial value vector is ${S_0, I_0, R_0, C_0, O_0}$, where $S_0 = 1 - I_0 - R_0 - C_0$, and $O_0$ is the number of revealed cases at the beginning. The initial values $I_0$, $R_0$ and $C_0$ are guessed during optimization process.

In [ ]:
test_sir_sco_kc = corona.CovidModel(country, model='sir_sco_kc')
test_sir_sco_kc.run(300)

## Model with incubation period and no immunity (SEIRS) <a name="seirs"/>

**SEIRS-O** is SIR model with incubation period and no immunity (recovered turn back into susceptible).

This model is modified [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model): 
- **S** is *Succeptible*
- **E** is *Exposed*, i.e. people who are infected, but not yet infectious.
- **I** is *Infectious*
- **R** is *Recovered* (and *diseased*). While in **R** state people are immune.
- **O** is *cOnfirmed* individuals, both hospitalized and quorantined at home with a positive test

The major feature of this model is conversion of some or recovered back into succeptible.

The model utilises the following parameters: 
- $\alpha$ models the speed of infection spread,
- $\beta$ represents the speed of recovery (and deaths),
- $e_i$ represents the speed of becoming infectious,
- $e_i$ represents the speed of becoming immune without becoming infectious,
- $r_s$ is the ratio of recovered people who are susceptible again,
- $k$ is the reveal factor, how many infectious people are confirmed as positive.

The differential equations:<br>
$\frac{dS}{dt} = -\alpha S I + r_s R$,<br>
$\frac{dE}{dt} = \alpha S I - e_i E - e_r E$, <br>
$\frac{dI}{dt} = e_i E - \beta I$, <br>
$\frac{dR}{dt} = \beta I + e_r E - r_s R$,<br>
$\frac{dO}{dt} = k I$.

The initial value vector is ${S_0, E_0, I_0, R_0, O_0}$, where $S_0 = 1 - E_0 - I_0 - R_0$ and $O_0$ is the number of revealed cases at the beginning. The initial values $E_0$, $I_0$ and $R_0$ are guessed during optimization process.


In [ ]:
test_seir_so = corona.CovidModel(country, model='seir_so')
test_seir_so.run(300)